```
bryan@ichabod mac_bucket % for file in *_survey.json; do
  databricks fs cp "$file" dbfs:/Volumes/geodata/petra/survey/
done
```


In [ ]:
df = spark.read.format("json").load("/Volumes/geodata/petra/survey/")

display(df)

In [ ]:
from common.transforms import string_to_iso_date, generate_hash
import pyspark.sql.functions as F

df_flat = df.select(
    F.col("repo_id"),
    F.col("uwi.uwi").alias("uwi"),
    F.col("uwi.wsn").alias("wsn"),
    F.col("dirsurvdata.active").alias("active"),
    F.col("dirsurvdata.adddate").alias("app_row_created"),
    F.col("dirsurvdata.chgdate").alias("app_row_changed"),
    # F.col("dirsurvdata.data").alias('data'),
    F.col("dirsurvdata.datasize").alias("datasize"),
    F.col("dirsurvdata.depunits").alias("depth_units"),
    F.col("dirsurvdata.dippresent").alias("dippresent"),
    F.col("dirsurvdata.md1").alias("md_top"),
    F.col("dirsurvdata.md2").alias("md_base"),
    F.col("dirsurvdata.numrecs").alias("numrecs"),
    F.col("dirsurvdata.remarks").alias("remarks"),
    F.col("dirsurvdata.tvd1").alias("tvd_top"),
    F.col("dirsurvdata.tvd2").alias("tvd_base"),
    F.col("dirsurvdata.xoff1").alias("x_offset_top"),
    F.col("dirsurvdata.xoff2").alias("x_offset_base"),
    F.col("dirsurvdata.yoff1").alias("y_offset_top"),
    F.col("dirsurvdata.yoff2").alias("y_offset_base"),
    F.col("dirsurvdef.survey_type").alias("survey_type"),
)

date_columns = ["app_row_created", "app_row_changed"]

df_survey = df_flat


# enforce timestamp for dates
for col_name in date_columns:
    df_survey = string_to_iso_date(df_survey, col_name, col_name)

# add id hash
id_columns = ["repo_id", "uwi"]
df_survey = generate_hash(df_survey, "id", "survey", *id_columns)


display(df_survey)